In [ ]:
!pip install openai
!pip install langchain_openai

In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    api_key="KEY",
)


completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are a smart home assistant. Your task is generating Trigger-Action Rules. After that, convert it to Linear Temporal Logic (LTL). Use this as an example: G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)"},
    {"role": "user", "content": "I want to turn off lights when I am not at home"}
  ]
)



print(completion.choices[0].message)

TAP RULES GENERATING

In [2]:
import os
import subprocess
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

os.environ['OPENAI_API_KEY'] = "KEY"

def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

# List the devices
devices_output = list_smartthings_devices()

prompt_template = ChatPromptTemplate.from_template(
    """You are a smart home assistant. The devices are in {devices_output}. 
    Your task is generating Trigger-Action Rules. After that, convert it to Linear Temporal Logic (LTL). 
    Use this as an example: G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)
    """
)

output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

chain = (
    prompt_template 
    | model 
    | output_parser
)

# Preparing the input data for the chain
input_data = {
    "devices_output": devices_output,
    "user_input": "Turn off all switch when motion sensor is inactivate"
}

# Merging devices_output and user input into a single prompt string
final_input = f"The devices are in {input_data['devices_output']}. {input_data['user_input']}"

# Invoking the chain with the final input
result = chain.invoke(final_input)

print(result)


### Trigger-Action Rules

1. **Trigger:** Motion Sensor is inactive
   **Action:** Turn off Aeotec Outlet 1

2. **Trigger:** Motion Sensor is inactive
   **Action:** Turn off Centralite Outlet 1

3. **Trigger:** Motion Sensor is inactive
   **Action:** Turn off Schema SmartApp Switch

### Linear Temporal Logic (LTL)

1. **Rule for Aeotec Outlet 1:**
   \( G (\neg \text{motion\_sensor\_active} \rightarrow F \neg \text{switch\_on\_Aeotec\_Outlet\_1}) \)

2. **Rule for Centralite Outlet 1:**
   \( G (\neg \text{motion\_sensor\_active} \rightarrow F \neg \text{switch\_on\_Centralite\_Outlet\_1}) \)

3. **Rule for Schema SmartApp Switch:**
   \( G (\neg \text{motion\_sensor\_active} \rightarrow F \neg \text{switch\_on\_Schema\_SmartApp\_Switch}) \)

### Explanation

- \( G \) stands for "Globally," meaning the rule should always be true.
- \( F \) stands for "Finally," meaning that the action should eventually happen.
- \(\neg \) stands for "not," indicating the negation of a condition.

So

PROPERTIES GENERATING

In [ ]:
import os
import subprocess
import csv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Set the OpenAI API key
os.environ['OPENAI_API_KEY'] = "KEY"

# Function to list smartthings devices
def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

# List the devices
devices_output = list_smartthings_devices()

# Create a prompt template
prompt_template = ChatPromptTemplate.from_template(
    """You are a smart home assistant. The devices are in {devices_output}. 
    Your task is generating Properties from user input. After that, convert it to Linear Temporal Logic (LTL). 
    Use this as an example: G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)
    """
)

# Define output parser and model
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

# Create the chain
chain = (
    prompt_template 
    | model 
    | output_parser
)

# Function to generate LTL and write to CSV
def generate_and_save_ltl(user_input, csv_filename='ltl_properties.csv'):
    # Preparing the input data for the chain
    input_data = {
        "devices_output": devices_output,
        "user_input": user_input
    }

    # Merging devices_output and user input into a single prompt string
    final_input = f"The devices are in {input_data['devices_output']}. {input_data['user_input']}"

    # Invoking the chain with the final input
    result = chain.invoke(final_input)

    # Write the result to a CSV file
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([user_input, result])

    return result

# Example usage
user_input = "When the motion sensor is active, turn on the switches"
ltl_property = generate_and_save_ltl(user_input)

print(ltl_property)


================== NEW MAIN VERSION ===========================

In [3]:
import os
import subprocess
import csv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

os.environ['OPENAI_API_KEY'] = "KEY"


In [4]:
def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

# List the devices
devices_output = list_smartthings_devices()
print(devices_output)


[
    {
        "deviceId": "7b66afce-2f5a-4bc4-b55f-6b949389f86d",
        "ownerId": "4ab9d1e8-e9e7-9a7f-c6f8-c62ec3ae220c",
        "name": "metering-switch",
        "label": "Aeotec Outlet 1",
        "deviceManufacturerCode": "0086-0103-0060",
        "manufacturerName": "SmartThingsCommunity",
        "deviceModel": "0103-0060",
        "presentationId": "a15343f1-ecbc-3bb5-9087-20f48fee9047",
        "locationId": "0f4f8426-8724-4501-aa02-06f415c7aa14",
        "roomId": "2f26d22c-ae2d-4c70-bc14-3e7e0d2ab250",
        "sharedLocations": [],
        "components": [
            {
                "id": "main",
                "label": "main",
                "capabilities": [
                    {
                        "id": "switch",
                        "version": 1
                    },
                    {
                        "id": "powerMeter",
                        "version": 1
                    },
                    {
                        "id": "energyMet

In [5]:
# Prompt templates
trigger_action_prompt_template = ChatPromptTemplate.from_template(
    """You are a smart home assistant. The devices are in {devices_output}. 
    Your task is generating Trigger-Action Rules. After that, convert it to Linear Temporal Logic (LTL). 
    Use this as an example: G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)
    """
)

property_prompt_template = ChatPromptTemplate.from_template(
    """You are a smart home assistant. The devices are in {devices_output}. 
    Your task is generating Properties from user input. After that, convert it to Linear Temporal Logic (LTL). 
    Use this as an example: G (motion_sensor_active -> F switch_on_Aeotec_Outlet_1)
    """
)

output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

trigger_action_chain = (
    trigger_action_prompt_template 
    | model 
    | output_parser
)

property_chain = (
    property_prompt_template 
    | model 
    | output_parser
)


In [55]:
def generate_trigger_action_rule(user_input):
    input_data = {
        "devices_output": devices_output,
        "user_input": user_input
    }

    final_input = f"The devices are in {input_data['devices_output']}. {input_data['user_input']}"

    result = trigger_action_chain.invoke(final_input)

    return result

user_input = "When the motion sensor detect motion, turn on the Aeotec switch"
trigger_action_rule = generate_trigger_action_rule(user_input)

print(trigger_action_rule)


Sure, let's start by generating the Trigger-Action rules and then convert them to Linear Temporal Logic (LTL).

### Trigger-Action Rule
1. **Trigger**: Motion sensor detects motion.
2. **Action**: Turn on the Aeotec switch (Aeotec Outlet 1).

### Linear Temporal Logic (LTL) Representation
The LTL formula that represents this behavior is:

\[ G (\text{motion\_sensor\_motion\_detected} \rightarrow F \text{switch\_on\_Aeotec\_Outlet\_1}) \]

Where:
- \( G \) stands for "Globally," indicating that the rule applies at all times.
- \( \rightarrow \) stands for "implies."
- \( F \) stands for "Finally," indicating that at some point in the future, the action will occur if the trigger is true.

In plain English, this LTL formula means that globally, whenever the motion sensor detects motion, it will eventually cause the Aeotec switch to turn on.

### Detailed Explanation of the Formula
- **Globally (G)**: The rule should always be active.
- **Trigger (motion_sensor_motion_detected)**: The moti

In [60]:
def generate_and_save_ltl(user_input, trigger_action_rule, csv_filename='ltl_properties.csv'):
    # Preparing the input data for the chain
    input_data = {
        "devices_output": devices_output,
        "user_input": user_input
    }

    # Merging devices_output and user input into a single prompt string
    final_input = f"The devices are in {input_data['devices_output']}. {input_data['user_input']}"

    # Invoking the chain with the final input
    result = property_chain.invoke(final_input)

    # Write the result to a CSV file with UTF-8 encoding
    with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([trigger_action_rule, result])

    return result

# Example usage
user_input_for_property = "When the motion sensor detects motion, turn on the Centralite switch"
ltl_property = generate_and_save_ltl(user_input_for_property, trigger_action_rule)

print(ltl_property)


### Properties from User Input

1. **Motion Sensor**: The device that detects motion.
    - Device ID: `badd4ded-6183-4fb6-928c-179dcd6d1229`
    - Label: "Motion Sensor"
    - Capability: "motionSensor"

2. **Centralite Switch**: The switch that needs to be turned on.
    - Device ID: `03f87109-2da4-4376-8372-580372d9ded6`
    - Label: "Centralite Outlet 1"
    - Capability: "switch"
    
### Linear Temporal Logic (LTL)

The LTL formula to express that whenever the motion sensor detects motion, the Centralite switch should eventually be turned on can be written as:

\[ \mathbf{G} (\text{motion\_sensor\_active} \rightarrow \mathbf{F} \text{switch\_on\_Centralite\_Outlet\_1}) \]

### Explanation

- **G** (Globally): Specifies that the condition should hold at all times.
- **motion\_sensor\_active**: Represents the event when the motion sensor detects motion.
- **F** (Finally): Specifies that the condition should eventually hold.
- **switch\_on\_Centralite\_Outlet\_1**: Represents the ev

In [61]:
def compare_trigger_action_to_properties(trigger_action, properties_filename='ltl_properties.csv'):
    # Read the properties from the CSV file
    with open(properties_filename, mode='r') as file:
        reader = csv.reader(file)
        properties = [row[1] for row in reader]
    
    # Check if the trigger action is in the properties
    if trigger_action in properties:
        return f"The Trigger-Action rule '{trigger_action}' is FOUND in the properties."
    else:
       # Regenerate the rule
        regenerated_rule = generate_trigger_action_rule(trigger_action)
        return f"The Trigger-Action rule '{trigger_action}' was NOT FOUND in the properties. Regenerated rule: {regenerated_rule}"

result = compare_trigger_action_to_properties(trigger_action_rule)

print(result)


The Trigger-Action rule 'Sure, let's start by generating the Trigger-Action rules and then convert them to Linear Temporal Logic (LTL).

### Trigger-Action Rule
1. **Trigger**: Motion sensor detects motion.
2. **Action**: Turn on the Aeotec switch (Aeotec Outlet 1).

### Linear Temporal Logic (LTL) Representation
The LTL formula that represents this behavior is:

\[ G (\text{motion\_sensor\_motion\_detected} \rightarrow F \text{switch\_on\_Aeotec\_Outlet\_1}) \]

Where:
- \( G \) stands for "Globally," indicating that the rule applies at all times.
- \( \rightarrow \) stands for "implies."
- \( F \) stands for "Finally," indicating that at some point in the future, the action will occur if the trigger is true.

In plain English, this LTL formula means that globally, whenever the motion sensor detects motion, it will eventually cause the Aeotec switch to turn on.

### Detailed Explanation of the Formula
- **Globally (G)**: The rule should always be active.
- **Trigger (motion_sensor_mot

=====================AutoGPT========================

=============================================

In [ ]:
import os
import subprocess
import csv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = "KEY"

def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

# List the devices
devices_output = list_smartthings_devices()

# Create the prompt
prompt = ChatPromptTemplate.from_template("You are a smart home assistant. The devices are in {devices_output}. Create a dataset with 100 linear temporal logic formulas for Trigger Action rules based on the {devices_output}")

# Initialize the model and output parser
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

# Create the chain
chain = (
    prompt 
    | model 
    | output_parser
)

input_data = {
    "devices_output": devices_output
}

# Get the result from the chain
result = chain.invoke(input_data)

# Split the result into individual LTL formulas if necessary (assumes result is a single string with formulas separated by newlines)
ltl_formulas = result.split('\n')

# Write the LTL formulas to a CSV file
csv_file_path = 'ltl_formulas.csv'
with open(csv_file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['LTL Formula'])  # Write header
    for formula in ltl_formulas:
        csvwriter.writerow([formula])

print(f"LTL formulas have been written to {csv_file_path}")


In [ ]:
import os
import subprocess
import csv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = "KEY"

def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

def generate_trigger_action_rule(devices_output):
    prompt = ChatPromptTemplate.from_template(
        "You are a smart home assistant. The devices are in {devices_output}. Create a Trigger-Action rule based on the devices in the SmartThings account.  Use the correct deviceId for the devices."
    )
    output_parser = StrOutputParser()
    model = ChatOpenAI(model="gpt-4o")

    chain = (
        prompt 
        | model 
        | output_parser
    )

    input_data = {
        "devices_output": devices_output
    }

    result = chain.invoke(input_data)
    return result.strip()  # Assuming the result is a single string for one rule

def generate_ltl_formulas(devices_output):
    prompt = ChatPromptTemplate.from_template(
        "You are a smart home assistant. The devices are in {devices_output}. Create a dataset with 100 linear temporal logic formulas for Trigger Action rules based on the {devices_output}."
    )
    output_parser = StrOutputParser()
    model = ChatOpenAI(model="gpt-4o")

    chain = (
        prompt 
        | model 
        | output_parser
    )

    input_data = {
        "devices_output": devices_output
    }

    result = chain.invoke(input_data)
    return result.split('\n')  # Assuming the result is a single string with formulas separated by newlines

# Rules Checking

def check_trigger_action_rule_with_ltl(trigger_action_rule, ltl_formulas):
    """
    Checks if the Trigger-Action rule matches any of the LTL formulas using LLM.

    :param trigger_action_rule: Trigger-Action rule
    :param ltl_formulas: List of LTL formulas
    :return: True if the rule matches any LTL formula, False otherwise
    """
    prompt = ChatPromptTemplate.from_template(
        "You are a smart home assistant. The following is a Trigger-Action rule and its corresponding LTL formulas. Check if the rule complies with any of the LTL formulas. Respond with 'True' if the rule complies with any formula, otherwise respond with 'False'.\n\nTrigger-Action Rule:\n{trigger_action_rule}\n\nLTL Formulas:\n{ltl_formulas}"
    )
    
    output_parser = StrOutputParser()
    model = ChatOpenAI(model="gpt-4o")

    chain = (
        prompt 
        | model 
        | output_parser
    )

    input_data = {
        "trigger_action_rule": trigger_action_rule,
        "ltl_formulas": "\n".join(ltl_formulas)
    }

    result = chain.invoke(input_data)
    return result.strip().lower() == 'true'

def save_to_csv(data, file_path, header):
    with open(file_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(header)  # Write header
        for item in data:
            csvwriter.writerow([item])
    print(f"Data has been written to {file_path}")

# Main loop
devices_output = list_smartthings_devices()
matched = False
iteration = 0

# Generate LTL formulas once
ltl_formulas = generate_ltl_formulas(devices_output)
print(f"Generated LTL Formulas:\n{ltl_formulas}")

while not matched and iteration < 10:  # Limit the iterations to avoid an infinite loop
    print(f"Iteration: {iteration+1}")
    trigger_action_rule = generate_trigger_action_rule(devices_output)
    print(f"Generated Trigger-Action Rule:\n{trigger_action_rule}")

    matched = check_trigger_action_rule_with_ltl(trigger_action_rule, ltl_formulas)

    if matched:
        print("Trigger-Action rule matches the LTL formulas.")
        save_to_csv([trigger_action_rule], 'trigger_action_rule.csv', ['Trigger Action Rule'])
    else:
        print("Trigger-Action rule does not match the LTL formulas. Generating new rule...")
    
    iteration += 1

# Save the LTL formulas to a CSV file
save_to_csv(ltl_formulas, 'ltl_formulas.csv', ['LTL Formula'])


In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from time import time

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = "KEY"


prompt = ChatPromptTemplate.from_template(
    """
    You are a smart home assistant. Create a Trigger-Action rule based on the user input.
        
    """
)


output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

chain = (
    {"request": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

In [ ]:
result = chain.invoke("Turn off all devices when nobody is home")

print(result)

In [ ]:
result = chain.invoke("When the front door is opened, turn on the hallway lights")

print(result)

In [ ]:
# Harper's code
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from time import time

os.environ['OPENAI_API_KEY'] = "key"

import subprocess
from openai import OpenAI

def list_smartthings_devices():
    result = subprocess.run(['smartthings', 'devices'], capture_output=True, text=True)
    return result.stdout

# List the devices
devices_output = list_smartthings_devices()

# prompt = ChatPromptTemplate.from_template("You are a smart home assistant. The devices are in {devices_output}. Your task is generating Trigger-Action Rules for Home Automation")
prompt = ChatPromptTemplate.from_template("You are a smart home assistant. The devices are in {devices_output}. Create some Trigger-Action rules based on the devices in the SmartThings account. Use the correct deviceId for the devices")

output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

chain = (
    prompt 
    | model 
    | output_parser
)

input_data = {
    "devices_output": devices_output
}

result = chain.invoke(input_data)

print(result)

NuSMV

In [1]:
import os
import subprocess
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

os.environ['OPENAI_API_KEY'] = "KEY"

user_inputs = [
    "The door should never be unlocked when no one is home.",
    "The thermostat should not exceed 25 degrees.",
    "Lights should be on when someone is in the room."
]

prompt_template = ChatPromptTemplate.from_template(
    """
    You are an assistant that helps generate NuSMV models.Generate NuSMV rules for the following user input: {user_input}
    """
)

output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o")

chain = (
    prompt_template 
    | model 
    | output_parser
)

input_data = {
    "user_input": "Never unlock doors when nobody is home"
}


result = chain.invoke(input_data)

print(result)


To create a NuSMV model based on the requirement "Never unlock doors when nobody is home," we need to define the states and transitions that capture this behavior. In NuSMV, we utilize modules, variables, and specifications to describe the system. Let's break down the components:

1. **Variables**:
   - `door_locked`: A boolean variable indicating whether the door is locked.
   - `someone_home`: A boolean variable indicating whether someone is home.

2. **Transitions**:
   - Define how the states of `door_locked` and `someone_home` can change over time.

3. **Specification**:
   - Ensure that the door is never unlocked (`door_locked = FALSE`) when nobody is home (`someone_home = FALSE`).

Here's how you can define these rules in NuSMV:

```nusmv
MODULE main
VAR
  door_locked : boolean;
  someone_home : boolean;

ASSIGN
  -- Initial states
  init(door_locked) := TRUE; -- Assume the door starts locked
  init(someone_home) := FALSE; -- Assume nobody is home initially

  -- Define transiti